In [1]:
# Importing Required Libraries
import os
import pandas as pd
import mysql.connector

In [2]:
# MySQL database configuration
# MYSQL_HOST = "localhost"
# MYSQL_USER = "root"
# MYSQL_PASSWORD = "Password!!"
MYSQL_HOST = "localhost"
MYSQL_USER = "mh30f"
MYSQL_PASSWORD = "Moh@1977"

In [3]:
def main():
    try:
        # Connect to MySQL
        connection = mysql.connector.connect(
            host=MYSQL_HOST,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            auth_plugin="mysql_native_password"
        )
        cursor = connection.cursor()

        # Specify the path to your Resources directory
        resources_dir = "C:\\Users\\mh30f\\OneDrive\\Documents\\GitHub\\YDOC2MySQL\\Resources"  # Update this path as needed
        files = [f for f in os.listdir(resources_dir) if f.endswith('.csv')]

        for file_name in files:
            # Read CSV into DataFrame, skipping rows 2 and 3
            file_path = os.path.join(resources_dir, file_name)
            df = pd.read_csv(file_path, skiprows=[1, 2], header=0)
            print(f"Data loaded from CSV {file_name}:\n{df.head()}")

            # Clean column names
            df.columns = df.columns.str.strip().str.replace(' ', '_')

            # Convert 'Date' and 'Time' columns into a single 'DateTime' column
            if 'Date' in df.columns and 'Time' in df.columns:
                df['Date'] = df['Date'].astype(str)
                df['Time'] = df['Time'].astype(str)

                # Create the 'DateTime' column by combining 'Date' and 'Time'
                df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce', dayfirst=True)

                # Drop the original 'Date' and 'Time' columns as they are no longer needed
                df.drop(columns=['Date', 'Time'], inplace=True)

                # Check for invalid datetime values and replace them with None
                df['DateTime'] = df['DateTime'].where(df['DateTime'].notna(), None)

                # Reorder the columns to make 'DateTime' the first column
                cols = ['DateTime'] + [col for col in df.columns if col != 'DateTime']
                df = df[cols]
                
                    # Drop rows and columns with all NaN values
                df = df.dropna(axis=1, how='all')
                df = df.fillna('MISSING')
                
                
            print(f"NAN column removed and Data with 'DateTime' column as the first column for {file_name}:\n{df}")

            # Create database and table
            db_name = os.path.basename(os.path.dirname(resources_dir))
            table_name = os.path.splitext(file_name)[0]

            cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
            cursor.execute(f"USE {db_name}")

            # Create or recreate the table
            cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")  # Drop the table if it already exists
            create_table_query = f"CREATE TABLE `{table_name}` ("
            for col in df.columns:
                dtype = 'FLOAT'
                if col == 'DateTime':
                    dtype = 'DATETIME DEFAULT NULL'
                elif pd.api.types.is_integer_dtype(df[col]):
                    dtype = 'INT'
                elif pd.api.types.is_string_dtype(df[col]):
                    dtype = 'VARCHAR(255)'
                create_table_query += f"`{col}` {dtype}, "
            create_table_query = create_table_query.rstrip(', ') + ")"
            cursor.execute(create_table_query)
            print(f"Table `{table_name}` created successfully.")

            # Batch insert rows into the database
            rows = [tuple(row) for row in df.itertuples(index=False, name=None)]
            placeholders = ', '.join(['%s'] * len(df.columns))
            insert_query = f"INSERT INTO `{table_name}` ({', '.join(f'`{col}`' for col in df.columns)}) VALUES ({placeholders})"
            cursor.executemany(insert_query, rows)
            print(f"Data inserted into `{table_name}` successfully.")

            # Export processed data to a new CSV file
            output_file_path = os.path.join(resources_dir, f"processed_{file_name}")
            df.to_csv(output_file_path, index=False)
            print(f"Processed data exported to {output_file_path}")

        # Commit the transaction
        connection.commit()

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [4]:
if __name__ == "__main__":
    main()

Data loaded from CSV YDOC_YD000101_W0041237_CT2X_SDI12_124130467_240925_030006.csv:
         Date     Time  Water Level      Temp  Barometric pressure  \
0  25/09/2024  0:15:00       13.599  31414.99             31395.36   
1  25/09/2024  0:30:00       13.600  30987.75             30964.87   
2  25/09/2024  0:45:00       13.600  30351.10             30328.69   
3  25/09/2024  1:00:00       13.600  30819.11             30796.35   
4  25/09/2024  1:15:00       13.599  30680.76             30661.59   

   Compensated Pressure  Casing temperature  Min voltage  EXT POWER / SOLAR  
0                28.673               3.318       10.281               20.0  
1                28.906               3.321       10.279               20.0  
2                28.906               3.322       10.279               19.9  
3                28.906               3.323       10.277               19.8  
4                28.673               3.323       10.277               19.6  
NAN column removed and Data

In [6]:
# Define your MySQL connection parameters
MYSQL_HOST = "localhost"
MYSQL_USER = "mh30f"
MYSQL_PASSWORD = "Moh@1977"
MYSQL_DATABASE = 'ydoc2mysql'

try:
    # Connect to MySQL
    connection = mysql.connector.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE,
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    # Fetch data from the table to verify
    select_query = "SELECT * FROM `YDOC_YDS00057_W7020330_4G_STS_BARO_125720519_240925_060006`"
    cursor.execute(select_query)
    rows = cursor.fetchall()

    # Display the fetched data
    for row in rows:
        print(row)

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the cursor and connection if they were created
    if cursor:
        cursor.close()
    if connection:
        connection.close()

(datetime.datetime(2024, 9, 25, 0, 15), 14.844, 17.15, 10.399, 4.445, 11.4, 3.16, 0.207)
(datetime.datetime(2024, 9, 25, 0, 30), 14.844, 17.15, 10.395, 4.449, 11.4, 3.3, 0.203)
(datetime.datetime(2024, 9, 25, 0, 45), 14.844, 17.15, 10.396, 4.448, 11.4, 3.3, 0.199)
(datetime.datetime(2024, 9, 25, 1, 0), 14.844, 17.15, 10.395, 4.449, 11.3, 3.3, 0.197)
(datetime.datetime(2024, 9, 25, 1, 15), 14.844, 17.15, 10.395, 4.449, 11.2, 3.31, 0.199)
(datetime.datetime(2024, 9, 25, 1, 30), 14.841, 17.15, 10.394, 4.447, 11.1, 3.3, 0.201)
(datetime.datetime(2024, 9, 25, 1, 45), 14.841, 17.15, 10.394, 4.447, 11.1, 3.3, 0.198)
(datetime.datetime(2024, 9, 25, 2, 0), 14.841, 17.15, 10.392, 4.449, 11.0, 3.3, 0.204)
(datetime.datetime(2024, 9, 25, 2, 15), 14.841, 17.15, 10.395, 4.446, 10.9, 3.3, 0.193)
(datetime.datetime(2024, 9, 25, 2, 30), 14.838, 17.15, 10.393, 4.445, 10.9, 3.3, 0.199)
(datetime.datetime(2024, 9, 25, 2, 45), 14.838, 17.15, 10.394, 4.444, 10.8, 3.3, 0.198)
(datetime.datetime(2024, 9, 25, 